In [ ]:
"""Importing the data for building the datasets from """
# # !pip install friends


# train_info = pd.read_csv('./data/wikilarge/grade_ratio_wiki_train.csv')
# val_info = pd.read_csv('./data/wikilarge/grade_ratio_wiki_val.csv')
# test_info = pd.read_csv('./data/wikilarge/grade_ratio_wiki_test.csv')

# with open('./data/wikilarge/wiki_train.src', 'r', encoding='utf-8') as f:
#     train_src = pd.DataFrame(f.readlines(), columns=['src'])
# with open('./data/wikilarge/wiki_val.src', 'r', encoding='utf-8') as f:
#     val_src = pd.DataFrame(f.readlines(), columns=['src'])
# with open('./data/wikilarge/wiki_test.src', 'r', encoding='utf-8') as f:
#     test_src = pd.DataFrame(f.readlines(), columns=['src'])

# with open('./data/wikilarge/wiki_train.tgt', 'r', encoding='utf-8') as f:
#     train_tgt = pd.DataFrame(f.readlines(), columns=['tgt'])
# with open('./data/wikilarge/wiki_val.tgt', 'r', encoding='utf-8') as f:
#     val_tgt = pd.DataFrame(f.readlines(), columns=['tgt'])
# with open('./data/wikilarge/wiki_test.tgt', 'r', encoding='utf-8') as f:
#     test_tgt = pd.DataFrame(f.readlines(), columns=['tgt'])

# train_data = pd.concat([train_info, train_src, train_tgt], axis=1)
# val_data = pd.concat([val_info, val_src, val_tgt], axis=1)
# test_data = pd.concat([test_info, test_src, test_tgt], axis=1)

# all_data = pd.concat([train_data, val_data, test_data], axis=0)
# # subset to src tgt and grade
# all_data = all_data[['src', 'tgt', 'abs_tgt_FKGL_Grade', 'abs_src_FKGL_Grade']]
# # drop duplicates
# all_data = all_data[all_data['src'] != all_data['tgt']]

In [8]:
"""For building the pickle files for each grade from original CSV files"""
from sklearn.model_selection import train_test_split
from collections import defaultdict
from datasets import DatasetDict, Dataset
import pickle
import pandas as pd

all_data = pd.read_csv('data/wikilarge/intermediate/wiki_cleaned_doubled.csv')
print(f"Loaded data shape: {all_data.shape}")

groups = all_data.groupby('abs_tgt_FKGL_Grade')

group_dict = defaultdict()
split_dict = defaultdict()
for group in groups:
    group_dict[group[0]] = group[1].reset_index(drop=True)

for group in group_dict.keys():
    train, test_val = train_test_split(group_dict[group], test_size=1024, random_state=42,)
    train_size = (train.shape[0] // 32 ) * 32
    assert train_size % 32 == 0, "Train size must be a multiple of 32"
    train = train.iloc[:train_size]  # ensure train size is a multiple of 32
    print(f"Group {group} - Train size: {train.shape[0]}, Test/Val size: {test_val.shape[0]}")
    val, test = train_test_split(test_val, test_size=0.5, random_state=42)
    split_dict[group] = {'train': train.reset_index(drop=True), 'val': val.reset_index(drop=True), 'test': test.reset_index(drop=True)}

# convert dataframes to datasets and save to pickle format
for group in split_dict.keys():
    train_data = Dataset.from_pandas(split_dict[group]['train'])
    val_data = Dataset.from_pandas(split_dict[group]['val'])
    test_data = Dataset.from_pandas(split_dict[group]['test'])
    dataset_dict = DatasetDict({'train': train_data, 'val': val_data, 'test': test_data})
    print(dataset_dict)
    print(f"Dataset for group {group} created with lengths: train={len(train_data)}, val={len(val_data)}, test={len(test_data)}")
    with open(f'data/wikilarge/intermediate/grade_{group}.pkl', 'wb') as f:
        pickle.dump(dataset_dict, f)


Loaded data shape: (292194, 36)
Group 0 - Train size: 960, Test/Val size: 1024
Group 1 - Train size: 896, Test/Val size: 1024
Group 2 - Train size: 4832, Test/Val size: 1024
Group 3 - Train size: 5984, Test/Val size: 1024
Group 4 - Train size: 18016, Test/Val size: 1024
Group 5 - Train size: 13856, Test/Val size: 1024
Group 6 - Train size: 30080, Test/Val size: 1024
Group 7 - Train size: 23552, Test/Val size: 1024
Group 8 - Train size: 31456, Test/Val size: 1024
Group 9 - Train size: 22016, Test/Val size: 1024
Group 10 - Train size: 32576, Test/Val size: 1024
Group 11 - Train size: 18016, Test/Val size: 1024
Group 12 - Train size: 23584, Test/Val size: 1024
Group 13 - Train size: 51808, Test/Val size: 1024
DatasetDict({
    train: Dataset({
        features: ['src', 'tgt', 'current_line', 'New Line', 'Line', 'abs_src_Length', 'abs_src_MaxDepDepth', 'abs_src_MaxDepLength', 'abs_src_DiffWords', 'abs_src_Leven', 'abs_src_WordCount', 'abs_tgt_Length', 'abs_tgt_MaxDepDepth', 'abs_tgt_MaxDep

In [38]:
"""Import pickle, convert back to dataframe, export as json in alpaca format"""
import os
from datasets import DatasetDict
import pickle
import pandas as pd
from huggingface_hub import login, HfApi
from dotenv import load_dotenv

load_dotenv()
token = os.getenv("HUGGINGFACE_HUB_TOKEN")
login(token=token)

def reimport_and_save_as_json(grade):
    # Load the pickled dataset

    #system_prompt = "Please rewrite the following sentence to make it easily understandable by students in Grade {tgt_ideal_Grade}. Ensure that the rewritten sentence is grammatically correct, fluent, and retains the core message of the original sentence without changing its meaning."
    instruction_prompt = "Rewrite this Input sentence to make it easily understandable by students in Grade {tgt_ideal_Grade}"# while preserving the meaning: Please note, if the initial rewrite does not meet the specified grade level, you are encouraged to modify and regenerate the output until the criteria are satisfactorily met. The final output should only include the last, correct version of the rewritten sentence(s)."
    
    with open(f'./data/wikilarge/intermediate/grade_{grade}.pkl', 'rb') as file:
        data = pickle.load(file)
    
    dataset = DatasetDict()
    
    for split_str in ["train", "val", "test"]:
        split = pd.DataFrame(data[split_str])
        split['input'] = split['src'].apply(lambda x: x.strip())
        split['output'] = split['tgt'].apply(lambda x: x.strip())

        #split['system'] = split.apply(lambda x: system_prompt.format(tgt_ideal_Grade=grade), axis=1)
        split['instruction'] = split.apply(lambda x: instruction_prompt.format(tgt_ideal_Grade=grade), axis=1)
  
        split['src_grade'] = split['abs_src_FKGL_Grade']
        split['tgt_grade'] = split['abs_tgt_FKGL_Grade']

        split = split[['input', 'output', 'instruction']]#, 'system']]#, 'src_grade', 'tgt_grade']]
        if split_str == 'val':
            split_str = 'validate'
        
        dataset[split_str] = Dataset.from_pandas(split)
    
    # Save as Alpaca-formatted JSON
    path = f'data/wikilarge/cleaned_graded_splits/wikilarge_grade{grade}_alpaca'
    os.makedirs(path, exist_ok=True)
    for split_str in ["train", "validate", "test"]:
        pd.DataFrame(dataset[split_str]).to_json(f'{path}/{split_str}.json', orient='records', indent=2)
    
    # Upload JSON files directly to hub to maintain JSON format
    api = HfApi()
    repo_id = f'williamplacroix/wikilarge_grade{grade}_alpaca'
    
    # # Create repo if it doesn't exist
    try:
        api.create_repo(repo_id=repo_id, private=True, exist_ok=True, repo_type="dataset")
    except Exception as e:
        print(f"Repo creation note: {e}")
    
    # Upload each JSON file
    for split_str in ["train", "validate", "test"]:
        try:
            api.delete_file(
                path_in_repo=f'{split_str}.json',
                repo_id=repo_id,
                repo_type="dataset",  # or "model"
            )
        except Exception as e:
            print(f"nothing to delete: {e}")
        api.upload_file(
            path_or_fileobj=f'{path}/{split_str}.json',
            path_in_repo=f'{split_str}.json',
            repo_id=repo_id,
            commit_message=f"Update {split_str} split",
            repo_type="dataset"
        )
    
    print(f"Grade {grade} processed and saved to {path} and uploaded to {repo_id}")

from tqdm import tqdm
# test with grade 8
for grade in tqdm({0,1,2,3,4,5,6,7,8,9,10,11,12,13}):
    reimport_and_save_as_json(grade)

  7%|▋         | 1/14 [00:04<00:53,  4.13s/it]

Grade 0 processed and saved to data/wikilarge/cleaned_graded_splits/wikilarge_grade0_alpaca and uploaded to williamplacroix/wikilarge_grade0_alpaca


 14%|█▍        | 2/14 [00:08<00:50,  4.22s/it]

Grade 1 processed and saved to data/wikilarge/cleaned_graded_splits/wikilarge_grade1_alpaca and uploaded to williamplacroix/wikilarge_grade1_alpaca


 21%|██▏       | 3/14 [00:14<00:54,  4.96s/it]

...

Grade 11 processed and saved to data/wikilarge/cleaned_graded_splits/wikilarge_grade11_alpaca and uploaded to williamplacroix/wikilarge_grade11_alpaca


 93%|█████████▎| 13/14 [04:20<00:24, 24.71s/it]

Grade 12 processed and saved to data/wikilarge/cleaned_graded_splits/wikilarge_grade12_alpaca and uploaded to williamplacroix/wikilarge_grade12_alpaca


train.json: 100%|██████████| 23.8M/23.8M [00:05<00:00, 4.37MB/s]
100%|██████████| 14/14 [05:02<00:00, 21.62s/it]

Grade 13 processed and saved to data/wikilarge/cleaned_graded_splits/wikilarge_grade13_alpaca and uploaded to williamplacroix/wikilarge_grade13_alpaca
